In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests 
import io
import seaborn as sns

In [ ]:
orig_members = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_members.csv')

In [ ]:
sorted_member_id = orig_members['member_id'].sort_values()
member_id_map ={}
start_id = 1
for id in sorted_member_id:
  member_id_map[id] = start_id
  start_id = start_id + 1

In [ ]:
orig_message_posts = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_message_posts.csv')

In [ ]:
orig_message_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13341 entries, 0 to 13340
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         13341 non-null  int64 
 1   msg_id             13341 non-null  int64 
 2   msg_topic_id       13341 non-null  int64 
 3   msg_date           13341 non-null  int64 
 4   msg_post           13341 non-null  object
 5   msg_post_key       13341 non-null  object
 6   msg_author_id      13341 non-null  int64 
 7   msg_ip_address     13341 non-null  object
 8   msg_is_first_post  13341 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 938.2+ KB


In [ ]:
orig_message_posts.drop(orig_message_posts.columns[[0,3, 4, 5, 7]], axis =1, inplace=True)

In [ ]:
orig_message_topics = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_message_topics.csv')
orig_message_topics.drop(orig_message_topics.columns[[0, 2, 4, 6, 7, 8, 14, 15, 16]], axis =1, inplace=True)

In [ ]:
adjMatrix = np.zeros((orig_members.shape[0], orig_members.shape[0]))

In [ ]:
print("Number of Unique IDs in orig_message_posts")
print(len(orig_message_posts['msg_author_id'].unique()))

print("Count of member_IDs present in orig_message_posts and orig_members.")
print(np.isin(orig_message_posts['msg_author_id'].unique(), sorted_member_id).sum())
# print(len(orig_message_posts.loc[orig_message_posts['msg_author_id'].isin(sorted_member_id)]))

print("Fraction of incorrect IDs")
print(1-(np.isin(orig_message_posts['msg_author_id'].unique(), sorted_member_id).sum()/len(orig_message_posts['msg_author_id'].unique())))

Number of Unique IDs in orig_message_posts
506
Count of member_IDs present in orig_message_posts and orig_members.
504
Fraction of incorrect IDs
0.0039525691699604515


In [ ]:
for id in orig_message_posts['msg_topic_id'].unique():
  df = orig_message_posts[orig_message_posts['msg_topic_id']==id]

  # the message was received by no one
  if len(df)<=1:
    continue

  else:

    # iterate over the sub-dataframe
    for i in range(len(df)):

      # if last row, pass
      if i==(len(df)-1):
        continue

      else:
        from_id = df.iloc[i+1]['msg_author_id']
        to_id = df.iloc[i]['msg_author_id']
        if to_id not in member_id_map or from_id not in member_id_map or from_id == to_id:
          continue
        from_mapping = member_id_map[from_id]
        to_mapping = member_id_map[to_id]
        # print(from_mapping, to_mapping)
        adjMatrix[from_mapping-1][to_mapping-1] +=1

In [ ]:
countZero = adjMatrix[np.where(adjMatrix == 0)].size
countTotal = adjMatrix.shape[0]*adjMatrix.shape[1]
countZero/countTotal

0.9958379783190104

In [ ]:
np.save("/content/drive/MyDrive/text gcn/adjMatrix/ironmarch/adjMatrix_last_reply_msg_posts", adjMatrix)

## Posts

Similar analysis.

In [ ]:
adjMatrix = np.zeros((orig_members.shape[0], orig_members.shape[0]))

In [ ]:
orig_posts = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_posts.csv')
orig_posts.drop(orig_posts.columns[[0,1, 2, 3,5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]], axis =1, inplace=True)

In [ ]:
orig_topics = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_topics.csv')

In [ ]:
print("Number of Unique IDs in orig_posts")
print(len(orig_posts['author_id'].unique()))

print("Count of member_IDs present in orig_posts and orig_members.")
print(np.isin(orig_posts['author_id'].unique(), sorted_member_id).sum())
# print(len(orig_message_posts.loc[orig_message_posts['msg_author_id'].isin(sorted_member_id)]))

print("Fraction of incorrect IDs")
print(1-(np.isin(orig_posts['author_id'].unique(), sorted_member_id).sum()/len(orig_posts['author_id'].unique())))

Number of Unique IDs in orig_posts
734
Count of member_IDs present in orig_posts and orig_members.
733
Fraction of incorrect IDs
0.0013623978201634523


In [ ]:
for id in orig_posts['topic_id'].unique():
  df = orig_posts[orig_posts['topic_id']==id]

  # the message was received by no one
  if len(df)<=1:
    continue

  else:

    # iterate over the sub-dataframe
    for i in range(len(df)):

      # if last row, pass
      if i==(len(df)-1):
        continue

      else:
        from_id = df.iloc[i+1]['author_id']
        to_id = df.iloc[i]['author_id']
        if to_id not in member_id_map or from_id not in member_id_map or from_id == to_id:
          continue
        from_mapping = member_id_map[from_id]
        to_mapping = member_id_map[to_id]
        # print(from_mapping, to_mapping)
        adjMatrix[from_mapping-1][to_mapping-1] +=1

## Sparsity

In [ ]:
countZero = adjMatrix[np.where(adjMatrix == 0)].size
countTotal = adjMatrix.shape[0]*adjMatrix.shape[1]
countZero/countTotal

0.9448476301555047

In [ ]:
np.save("/content/drive/MyDrive/text gcn/adjMatrix/ironmarch/adjMatrix_last_reply_posts", adjMatrix)